(dynamic_types)=
# Динамическая типизация 

Следующий код `python` исполняется без ошибок.

In [2]:
a = 0               # a - переменная типа int
print(a, type(a))
a = 0.              # теперь a - переменная типа float
print(a, type(a))
a = "zero"          # теперь a - переменная типа str
print(a, type(a))
a = [0, 0., "zero"] # теперь a - переменная типа list
print(a, type(a))

0 <class 'int'>
0.0 <class 'float'>
zero <class 'str'>
[0, 0.0, 'zero'] <class 'list'>


Кажется, что переменная `a` меняет свой тип 3 раза. Как это возможно? Прежде чем ответить на этот вопрос, вспомним, как это устроенно в `C/C++`.
```{note}
Функция [type](https://docs.python.org/3/library/functions.html#type) спрашивает у объекта его тип. 
```
## Статическая типизация в `C++`

Чтобы разобрать, как устроена типизация в `C/C++`, рассмотрим следующий код на языке `C/C++`.

```c++
#include<iostream>

int main(){
	// 1
	int a = 65 + 66*256 + 67*65536 + 68*16777216;
	std::cout << a << std::endl;
	
	// 2
	char *b = reinterpret_cast<char*>(&a);
	// 3
	for(size_t i=0; i<4; ++i)
		std::cout << b[i];
	std::cout << std::endl;
	
	// 4
	a = 'A';
	std::cout << a; 
	
	return 0;
}
```
Если его скомпилировать и запустить, то вы должны получить следующее в стандартном потоке вывода.
```sh
1145258561
ABCD
65
```
```{note}
Вообще говоря, код опирается на предположение, что в системе целочисленный тип `int` занимает 4 байта, что не гарантируется стандартами `C++`.
```

Разберем по шагам, что происходит в этой программе. 

### // 1

В начале объявляется целочисленная переменная `a` типа `int`, которая инициализируется значением $65 \cdot 256^0 + 66 \cdot 256^1 + 67 \cdot 256^{2} + 68 \cdot 256^{3} = 1145258561$. 
```c++
// 1
int a = 65 + 66*256 + 67*65536 + 68*16777216;
std::cout << a << std::endl;
```
В момент объявления выделяется достаточная для хранения переменной типа `int` (4 байта на большинстве платформ) область оперативной памяти, которая связывается с переменной `a`. Во момент инициализации в эту область памяти записывается число 1145258561. Это число намерено задано в разложении по степеням числа $256$, чтобы явно выделить байты этого числа: 65, 66, 67 и 68 (см. [статью в википедии](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%80%D1%8F%D0%B4%D0%BE%D0%BA_%D0%B1%D0%B0%D0%B9%D1%82%D0%BE%D0%B2)).  
### // 2

Далее объявляется указатель `b` на символьную переменную типа `char`, который инициализируется адресом памяти переменной `a`. 
```c++
// 2
char *b = reinterpret_cast<char*>(&a);
```
Чтобы провернуть трюк, необходимо сначала поменять тип указателя, для чего используется [reinterpret_cast](https://en.cppreference.com/w/cpp/language/reinterpret_cast), т.к. `C++` --- статические типизированный язык программирования и компилятор бы ругался на  

### // 3

Далее в цикле выводятся значения `b[0]`, `b[1]`, `b[2]` и `b[3]`.
```c++
// 3
for(size_t i=0; i<4; ++i)
	std::cout << b[i];
std::cout << std::endl;
```
В результате видим строку `ABCD`. Чтобы это понять, необходимо учесть 
- `b` --- указатель, который хранит в себе адрес переменной `a`;
- `b[0]` эквивалентно `*b` --- интерпретированная в качестве символьного типа `char` область памяти от указателя `b` размером 1 байт. 
- `b[i]` эквивалентно `*(b + i) `, т.е. тоже самое, что и `b[0]`, но на `i` байт правее.
- т.к. в и тоге выводится символьная переменная, то она выводится согласно таблице [ASCII](https://ru.wikipedia.org/wiki/ASCII), в которой 65 соответствует символу `A`, 66 --- символу `B`, 67 --- символу `C`, 68 --- символу `D`.

Следующий рисунок иллюстрирует примерное устройство памяти в этой программе на этот момент времени.
```{figure} /_static/lecture_specific/dynamic_typed_language/cpp.svg
```
### // 4

Далее в переменную `a` записывается записывается значение "`'A'`";
```c++
// 4
a = 'A';
std::cout << a; 
```
Т.к. компилятор знает, что `a` --- переменная типа `int` и `A` --- значение типа `char`, которое может быть безопасно без потерь приведено к `int`, то он допускает эту операцию. При этом в область памяти, соответствующую переменной `a`, записываются новые данные.

### Итого

Итого, опуская технические подробности:
- переменная соответствует области в памяти;
- то, как область памяти будет интерпретирована, определяется типом переменной, по которой к этой области памяти обращаются.
- оператор присвоения "`=`" записывает значение справа от него в область памяти, именованную переменной слева от него. 

## Динамическая типизация в `python`

В `python` тип хранится не в переменной, а в самом объекте, а сама переменная всего лишь ссылается на объект (в `python` всё объекты), ничего не подозревая о типе этого самого объекта. На самом деле переменная в `python` представляет собой обертку над указателем `C/C++`, который всегда автоматически разыменовывается, а оператор присвоения "`=`" связывает переменную слева от него с объектом справа от него (изменяет значение указателя).

```{note}
В связи с этим, иногда термин **"переменная"** (**variable**) избегается в отношении `python` и заменяется терминами "имя" (**name**), "ссылка" (**reference**) или идентификатор.  
```
```{note}
В python работать с адресами памяти напрямую невозможно. Единственное исключение составляет функция `id`, которая в некоторых реализациях возвращает адрес памяти (целое число), где хранится объект. Эта функция нужна только для того, чтобы всегда можно было выяснить, ссылаются ли два имени на один и тот же объект. Обратится по этому адресу средствами `python` невозможно. 
```

Рассмотрим следующий код в `python`.

In [1]:
a = 0       # 1
a = 42      # 2
a = "s"     # 3


Первая инструкция `a = 0`. Сначала вычисляется значение выражения справа: создается объект типа `int` содержащий в себе значение `0`. Так как имени `a` до этого объявлено не было, то такое имя создаётся и связывается с объектом справа.

In [ ]:
a = 0       # 1

```{figure} /_static/lecture_specific/dynamic_typed_language/a0.svg
```

Вторая инструкция `a = 42`. Создаётся объект типа `int`, содержащий значение `42`. Имя `a` связывается с новым объектом.  

In [ ]:
a = 42      # 2

```{figure} /_static/lecture_specific/dynamic_typed_language/a42.svg
  :name: ais42
```

Третья инструкция `a="s"`. Создаётся объект типа `str`, содержащий значение `"s"`. Имя `a` связывается связывается с новым объектом, который имеет другой тип. 

In [2]:
a = "s"     # 3

```{figure} /_static/lecture_specific/dynamic_typed_language/as.svg
```

```{note}
  Иногда говорят, что тип объекта "живёт" в самом объекте, а не в "переменной". 
```